In [2]:
import pandas as pd

In [40]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [4]:
insurance

,pid,tiv_2015,tiv_2016,lat,lon
0,1,10.0,5.0,10.0,10.0
1,2,20.0,20.0,20.0,20.0
2,3,10.0,30.0,20.0,20.0
3,4,10.0,40.0,40.0,40.0


Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:

* have the same tiv_2015 value as one or more other policyholders, and
* are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).

Round tiv_2016 to two decimal places.

The result format is in the following example.

| tiv_2016 |
|----------|
| 45.00    |

In [10]:
# have the same tiv_2015 value as one or more other policyholders, and
insurance2=insurance.groupby(by="tiv_2015")["tiv_2015"].size().reset_index(name="count")
insurance2

,tiv_2015,count
0,10.0,3
1,20.0,1


In [22]:
# are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).
insurance["loc"]=insurance["lat"].astype(str)+insurance["lon"].astype(str)
print(insurance)

loc_count=insurance.groupby(by="loc")["loc"].size().reset_index(name="count")
print(loc_count)

valid_loc = loc_count.loc[loc_count["count"] == 1, "loc"]
print(valid_loc)

   pid  tiv_2015  tiv_2016   lat   lon       loc
0    1      10.0       5.0  10.0  10.0  10.010.0
1    2      20.0      20.0  20.0  20.0  20.020.0
2    3      10.0      30.0  20.0  20.0  20.020.0
3    4      10.0      40.0  40.0  40.0  40.040.0
        loc  count
0  10.010.0      1
1  20.020.0      2
2  40.040.0      1
0    10.010.0
2    40.040.0
Name: loc, dtype: object


In [ ]:
tiv_2016 = insurance.loc[
    (insurance.tiv_2015.isin(insurance2.loc[insurance2["count"] > 1, "tiv_2015"])) &
    (insurance["loc"].isin(valid_loc))
, "tiv_2016"].sum()
pd.DataFrame([tiv_2016], columns=["tiv_2016"])

,tiv_2016
0,45.0


# Proper way (top answer)

In [47]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})
# Deduplicate based on tiv_2015 and mark duplicated.
duplicated_tv_2015=insurance.duplicated(['tiv_2015'],keep=False)
insurance['shared_tv']=duplicated_tv_2015
# Deduplicate based on location.
unique=~insurance.duplicated(subset=['lat','lon'],keep=False)
insurance['unique']=unique
print(insurance)
# Filter out.
insurance=insurance[(insurance['shared_tv']==True) & (insurance['unique']==True)]
newsum=insurance['tiv_2016'].sum()
final=pd.DataFrame({'tiv_2016':[newsum]})
final

   pid  tiv_2015  tiv_2016   lat   lon  shared_tv  unique
0    1      10.0       5.0  10.0  10.0       True    True
1    2      20.0      20.0  20.0  20.0      False   False
2    3      10.0      30.0  20.0  20.0       True   False
3    4      10.0      40.0  40.0  40.0       True    True


,tiv_2016
0,45.0
